# Setup Variables


In [140]:
aws_profile_name = "dev"
aws_region = "us-east-1"

In [141]:
from langchain.globals import set_verbose, set_debug

set_debug(False)
set_verbose(False)

# Setup AWS Bedrock Model


In [142]:
import boto3
from langchain_aws import ChatBedrockConverse

boto3.setup_default_session(profile_name=aws_profile_name)
BEDROCK_CLIENT = boto3.client("bedrock-runtime", aws_region)
model = ChatBedrockConverse(
    client=BEDROCK_CLIENT,
    model="anthropic.claude-3-5-sonnet-20240620-v1:0",
    max_tokens=1000,
    temperature=0.2,
    top_p=0.9,
)

# Define Our Tools


## Get Teammates By Name

Go to our "api" to get all teammates that have similar names. In this case, our API is actually just a JSON file we read in at runtime.


In [143]:
from langchain_core.tools import tool
from pydantic import BaseModel, TypeAdapter


class Teammate(BaseModel):
    id: int
    name: str
    favorite_color: str


@tool(parse_docstring=True)
def get_teammates_by_name(search_str: str) -> list[Teammate]:
    """Call to an API to get teammate details searching by their name.

    Args:
        search_str: The search string used to look for a teammate.
            This can be a partial string and will return all teammates
            who contain this search string as a substring, case insensitive.

    Returns:
        list[Teammate]: A list of any teammates with names containing the search string.
    """
    with open("data/teammates.json", "r") as file:
        teammates = TypeAdapter(list[Teammate]).validate_json(file.read())
    return [
        teammate
        for teammate in teammates
        if search_str.lower() in teammate.name.lower()
    ]

## Consolidate All Our Tools


In [144]:
from langchain_core.tools.simple import Tool
from typing import cast

tools = cast(list[Tool], [get_teammates_by_name])

# REWOO Parts


## The Plan

First thing we need to do is generate a plan to solve our problem. This requires more work than our continuous loop technique, but results in fewer calls and a more definite set of actions you can communicate to the user.

Lets define what we want a "Plan" object to look like. We are going to leverage Pydantic descriptions so we don't have to define this information in our prompt.


In [145]:
from typing import Any, Dict, List, Union
from pydantic import BaseModel, Field
from langchain_core.output_parsers.pydantic import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser


class Step(BaseModel):
    id: int = Field(
        description="Unique identifier for the step. IDs must be unique and in order."
    )
    summary: str = Field(
        description="Summary of the step in plain english for a human to read."
    )
    tool_name: str = Field(description="Name of the tool to be used in this step.")
    tool_input: str = Field(description="Input to the tool.")
    tool_output: Any = Field(
        None,
        description="Output from the tool, can be null, a string, or an object with unknown properties. Default is null.",
    )


class Plan(BaseModel):
    summary: str = Field(
        description="A summary of your plan in plain english for a human to read."
    )
    steps: List[Step] = Field(
        description="An ordered list of Step objects to complete the task."
    )

#### Also, lets quickly create some parsers that we can use later down the line for both prompting and failover


In [146]:
plan_parser = PydanticOutputParser(pydantic_object=Plan)
print(plan_parser.get_format_instructions())
parser = OutputFixingParser.from_llm(parser=plan_parser, llm=model)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Step": {"properties": {"id": {"description": "Unique identifier for the step. IDs must be unique and in order.", "title": "Id", "type": "integer"}, "summary": {"description": "Summary of the step in plain english for a human to read.", "title": "Summary", "type": "string"}, "tool_name": {"description": "Name of the tool to be used in this step.", "title": "Tool Name", "type": "string"}, "tool_input": {"description": "Input to the tool.", "title": "Tool Input", "type": "string"}, "tool_output": {"default": null, "description": "Outp

### Our Prompt to Generate Our Plan

We get to leverage Langchain parser and tools so we only have to manage definitions in once place.

Examples are good to throw in so it finds patterns to latch on to.


In [147]:
planning_prompt = f"""
A human is going to provide you with a task or message, make plans that can solve their problem step by step.

{plan_parser.get_format_instructions().replace("{","{{{{").replace("}","}}}}")},

When creating the plans, if you need something from a previous step you can reference it directly using '[stepId]'.
For example, if you need the output from step 1, you can reference it as '[1]'. If you need the output from step 2, you can reference it as '[2]', and so on.

For each plan, indicate which external tool together with tool input to retrieve evidence. You do not have to use any tools if you do not need to.
Make sure the plan is optimized and as few of steps as required to perform the given task.
You also have access to the history of the conversation with the user to help you plan effectively.

Here are some examples of the JSON schema with the user input and the task to be completed including available tools:
<examples>
Input: Are there multiple teammates with the name Lovelace?
Output Plan:
{{{{
  'summary': 'We need to get all teammates with the name Lovelace',
  'steps': [
    {{{{
      'id': 1,
      'summary': 'Search the teammates API for all teammates with the name Lovelace.',
      'tool_name': 'get_teammates_by_name',
      'tool_input': 'lovelace',
      'tool_output': null
    }}}}
  ]
}}}}

Input: 5x + 3 = 18, solve for x.
Output Plan:
{{{{
  'summary': 'Solve the equation 5x + 3 = 18 for x, we do not need any tools for this task.',
  'steps': []
}}}}

Input: Hello
Output Plan:
{{{{
  'summary': 'The user is just saying hello, we do not need any tools for this task.',
  'steps': []
}}}}
</examples>

Make sure to follow the schema strictly and provide the plan in the correct format. 
You will only respond with valid JSON.
Do not include any additional information or text in the response as it will be considered as an invalid response and you will be asked to provide the answer again.
You will never change the user's task or provide an answer to the task.
You will only provide a plan to solve the task.
DO NOT call tools that are not provided to you.
""".strip()

### Example of a getting a plan

NOTE: Sometimes we MAY get calls out to tools that do not exist. We should try and handle those gracefully. Below, I simply ignore the calls.

This could potentially be avoided by providing better tool info in our prompt, but for now, we are going to rely on what langchain is doing naturally during the `bind_tools` call.


In [166]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", planning_prompt),
        ("human", "Do Matt and Grey have the same favorite color?"),
    ]
)
planning_chain = prompt | model.bind_tools(tools) | parser
plan_output: Plan = planning_chain.invoke({})
print(plan_output.model_dump_json(indent=2))

{
  "summary": "To determine if Matt and Grey have the same favorite color, we need to search for information about both teammates.",
  "steps": [
    {
      "id": 1,
      "summary": "Search for teammates with the name Matt.",
      "tool_name": "get_teammates_by_name",
      "tool_input": "Matt",
      "tool_output": null
    },
    {
      "id": 2,
      "summary": "Search for teammates with the name Grey.",
      "tool_name": "get_teammates_by_name",
      "tool_input": "Grey",
      "tool_output": null
    }
  ]
}


# Solving

After our `Planning` and `Tool Calling` phases, we need to wrap everything up with a `Solve` phase. We will feed in the plan with all the outputs attached.


In [149]:
solve_prompt = """
Solve the following task or problem.
To solve the problem, we have made step-by-step Plan and retrieved corresponding output for each step.
Use them with caution since long output might contain irrelevant information.

<plan>
{plan}
</plan>

Now solve the question or task according to provided plan and output above.
Respond with the answer directly with no extra words unless they provide helpful detail.
If you don't know the answer, don't make something up,just tell what information you need to solve the problem or answer the message.
""".strip()

# Assembling our Graph


Overall we will want our final graph to do these things:

1. Make a plan
2. Iterate over steps in that plan and call tools until all tools have an `output`
3. Solve


## First, State Definition and Some Helper Methods

This defines what state we will be passing around our graph every time we need to execute or make a decision of where to go next.

In this case, we simply want to use the built in `MessagesState`, but add our plan object to it as well.


In [150]:
from enum import Enum
from langgraph.graph import MessagesState


class RewooState(MessagesState):
    plan: Plan


def get_next_step(state: RewooState):
    return next(
        (step for step in state["plan"].steps if step.tool_output is None), None
    )

## Now the Execution Methods. These will make our graph `nodes`.

All of these methods will receive our graph state and return and object that will be merged with our graph state. This can just be our modified full state again if that is easier.

NOTE: There is some code in our `call_tool` method that tries to reconcile prior tools outputs that are referenced in future tool inputs. This could use some love.


In [151]:
import re


def create_plan(state: RewooState):
    print("Planning")
    print(state)
    prompt = ChatPromptTemplate.from_messages(
        [("system", planning_prompt), *state["messages"]]
    )
    planning_chain = prompt | model.bind_tools(tools) | parser
    plan: Plan = planning_chain.invoke({})
    return {"plan": plan}


def call_tool(state: RewooState):
    print("Calling Tool")
    print(state)
    step = get_next_step(state)
    if step:
        tool = next((tool for tool in tools if tool.name == step.tool_name), None)
        if tool:

            def get_tool_output(match_obj):
                return str(
                    next(
                        (
                            step
                            for step in state["plan"].steps
                            if step.id == match_obj.group(1)
                        )
                    )
                )

            step.tool_input = re.sub(r"\[(\d+)\]", get_tool_output, step.tool_input)
            step.tool_output = tool.invoke(step.tool_input)
        else:
            step.tool_output = "Tool does not exist."
    return state


def solve(state: RewooState):
    print("Solving")
    print(state)
    prompt = ChatPromptTemplate.from_messages(
        [("system", solve_prompt), *state["messages"]]
    )
    solve_chain = prompt | model
    result = solve_chain.invoke(state)
    result.additional_kwargs["plan"] = state["plan"]
    return {"messages": [result]}

## Now Decision Making/Routing Methods. These will make our graph `edges`.

All of these methods will receive our graph state object and return a string that represents the next node that should be executed.

Routing methods are only needed when there is a `conditional` routing event. Some events you will see later on are simply hard coded since they always follow each other.


In [163]:
from typing import Literal


def should_call_tool_or_solve(state: RewooState):
    return call_tool.__name__ if get_next_step(state) else solve.__name__

## Now we put it all together!

NOTE: The ascii graph is not correct due to ENUM vs Str and not great Python typing


In [164]:
from langgraph.graph import StateGraph

graph = StateGraph(RewooState)

# definitions
graph.add_node(create_plan)
graph.add_node(call_tool)
graph.add_node(solve)

# navigation
graph.set_entry_point(create_plan.__name__)
graph.add_edge(create_plan.__name__, call_tool.__name__)
graph.add_conditional_edges(call_tool.__name__, should_call_tool_or_solve)
graph.set_finish_point(solve.__name__)

agent = graph.compile()
agent.get_graph().print_ascii()

        +-----------+     
        | __start__ |     
        +-----------+     
              *           
              *           
              *           
       +-------------+    
       | create_plan |    
       +-------------+    
              .           
              .           
              .           
        +-----------+     
        | call_tool |     
        +-----------+     
         ..        ..     
       ..            .    
      .               ..  
+-------+               . 
| solve |             ..  
+-------+            .    
         **        ..     
           **    ..       
             *  .         
         +---------+      
         | __end__ |      
         +---------+      


# Now that we have our agent assembled, we just need to seed it with some messages and get back results!


In [154]:
output = agent.invoke(
    {"messages": [("human", "Do Matt and Grey have the same favorite color?")]}
)

Planning
{'messages': [HumanMessage(content='Do Matt and Grey have the same favorite color?', additional_kwargs={}, response_metadata={}, id='f237a892-e691-454d-81bb-450fd7e1d3a3')]}
Calling Tool
{'messages': [HumanMessage(content='Do Matt and Grey have the same favorite color?', additional_kwargs={}, response_metadata={}, id='f237a892-e691-454d-81bb-450fd7e1d3a3')], 'plan': Plan(summary='To determine if Matt and Grey have the same favorite color, we need to search for their information in the teammates database and compare their favorite colors.', steps=[Step(id=1, summary="Search for Matt's information in the teammates database.", tool_name='get_teammates_by_name', tool_input='Matt', tool_output=None), Step(id=2, summary="Search for Grey's information in the teammates database.", tool_name='get_teammates_by_name', tool_input='Grey', tool_output=None)])}
Calling Tool
{'messages': [HumanMessage(content='Do Matt and Grey have the same favorite color?', additional_kwargs={}, response_met

## The Final Answer


In [155]:
print(output["messages"][-1].content)

No, Matt and Grey do not have the same favorite color. Matt's favorite color is green, while Grey's favorite color is purple.


## Message Details


In [156]:
import json

for message in output["messages"]:
    print(json.dumps(message, indent=2, default=lambda x: x.dict()))

{
  "content": "Do Matt and Grey have the same favorite color?",
  "additional_kwargs": {},
  "response_metadata": {},
  "type": "human",
  "name": null,
  "id": "f237a892-e691-454d-81bb-450fd7e1d3a3",
  "example": false
}
{
  "content": "No, Matt and Grey do not have the same favorite color. Matt's favorite color is green, while Grey's favorite color is purple.",
  "additional_kwargs": {
    "plan": {
      "summary": "To determine if Matt and Grey have the same favorite color, we need to search for their information in the teammates database and compare their favorite colors.",
      "steps": [
        {
          "id": 1,
          "summary": "Search for Matt's information in the teammates database.",
          "tool_name": "get_teammates_by_name",
          "tool_input": "Matt",
          "tool_output": [
            {
              "id": 4,
              "name": "Matt Vincent",
              "favorite_color": "green"
            }
          ]
        },
        {
          "id": 2